In [1]:
import torch
from transformer_lens import HookedTransformer
from functools import partial
import torch.nn.functional as F
from eap.metrics import logit_diff, direct_logit
import transformer_lens.utils as utils
from eap.graph import Graph
from eap.dataset import EAPDataset
from eap.attribute import attribute
import time
from rich import print as rprint
import pandas as pd
from eap.evaluate import evaluate_graph, evaluate_baseline,get_circuit_logits

/hpc2hdd/home/hchen763/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/hpc2hdd/home/hchen763/.conda/envs/knowledgecircuit/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/hpc2hdd/home/hchen763/.conda/envs/knowledgecircuit/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
LLAMA_2_7B_CHAT_PATH = "meta-llama/Llama-2-7b-chat-hf"
from transformers import LlamaForCausalLM
model = HookedTransformer.from_pretrained(LLAMA_2_7B_CHAT_PATH, device="cuda", fold_ln=False, center_writing_weights=False, center_unembed=False)
model.cfg.use_split_qkv_input = True
model.cfg.use_attn_result = True
model.cfg.use_hook_mlp_in = True

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.88s/it]


Loaded pretrained model meta-llama/Llama-2-7b-chat-hf into HookedTransformer


In [ ]:
clean_subject = 'Eiffel Tower'
corrupted_subject = 'the Great Walls'
clean = f'The official currency of the country where {clean_subject} is loacted in is the'
corrupted = f'The official currency of the country where {corrupted_subject} is loacted in is the'
assert len(model.to_str_tokens(clean.format(clean_subject))) == len(model.to_str_tokens(corrupted.format(corrupted_subject)))
labels = ['Euro','Chinese']
country_idx = model.tokenizer(labels[0],add_special_tokens=False).input_ids[0]
corrupted_country_idx = model.tokenizer(labels[1],add_special_tokens=False).input_ids[0]
# dataset = {k:[] for k in ['clean','country_idx', 'corrupted',  'corrupted_country_idx']}
# for k, v in zip(['clean', 'country_idx', 'corrupted', 'corrupted_country_idx'], [clean, country_idx, corrupted, corrupted_country_idx]):
#     dataset[k].append(v)
# df2 = pd.DataFrame.from_dict(dataset)
# df2.to_csv(f'capital_city.csv', index=False)

In [69]:
label = [[country_idx, corrupted_country_idx]]
label = torch.tensor(label)
data = ([clean],[corrupted],label)

In [ ]:
# ds = EAPDataset(filename='capital_city.csv',task='fact-retrieval')
# dataloader = ds.to_dataloader(1)

In [12]:
g = Graph.from_model(model)
start_time = time.time()
# Attribute using the model, graph, clean / corrupted data and labels, as well as a metric
attribute(model, g, data, partial(logit_diff, loss=True, mean=True), method='EAP-IG-case', ig_steps=100)
# attribute(model, g, data, partial(direct_logit, loss=True, mean=True), method='EAP-IG-case', ig_steps=30)
# attribute(model, g, dataloader, partial(logit_diff, loss=True, mean=True), method='EAP-IG', ig_steps=30)
g.apply_topn(5000, absolute=True)
g.prune_dead_nodes()

g.to_json('graph.json')

gz = g.to_graphviz()
gz.draw(f'graph.png', prog='dot')

end_time = time.time()
execution_time = end_time - start_time
print(f"程序执行时间：{execution_time}秒")

AssertionError: Attention mask shape torch.Size([1, 121]) does not match tokens shape torch.Size([1, 123])

In [72]:
logits = get_circuit_logits(model, g, data)
get_component_logits(logits, model, answer_token=model.to_tokens('Euro',prepend_bos=False)[0], top_k=5)

Performance on answer token:
Rank: 0        Logit: 16.94 Prob: 56.56% Token: |Euro|

Top 0th token. Logit: 16.94 Prob: 56.56% Token: |Euro|
Top 1th token. Logit: 15.96 Prob: 21.39% Token: |French|
Top 2th token. Logit: 14.06 Prob:  3.18% Token: |_|
Top 3th token. Logit: 13.95 Prob:  2.85% Token: |euro|
Top 4th token. Logit: 13.91 Prob:  2.74% Token: |Eu|


In [ ]:
baseline = evaluate_baseline(model, dataloader, partial(logit_diff, loss=False, mean=False)).mean().item()
results = evaluate_graph(model, g, dataloader, partial(logit_diff, loss=False, mean=False)).mean().item()
print(f"Original performance was {baseline}; the circuit's performance is {results}")

In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [20]:
model_path = "/hpc2hdd/home/hchen763/jhaidata/local_model/DeepSeek-R1-Distill-Qwen-1.5B"

# 1. 加载分词器和模型
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).to("cuda")

In [52]:
messages = [
    {"role": "system", "content": "You are a helpful assistant. You should generate answer as short as possible."},
    {"role": "user", "content": "Natalia sold clips to 50 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"}
]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", padding=True).to("cuda")
attention_mask = (inputs != tokenizer.pad_token_id).long()   # 通常如此，如果有 pad_token
outputs = model.generate(
    inputs,
    attention_mask=attention_mask,
    max_new_tokens=100,
    do_sample=False,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

response_ids = outputs[0]
response = tokenizer.decode(response_ids)
print(response)
# print(response_ids)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


<｜begin▁of▁sentence｜>You are a helpful assistant. You should generate answer as short as possible.<｜User｜>Natalia sold clips to 50 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? Please reason step by step, and put the answer as short as possible.
</think>

Natalia sold 50 clips in April and half as many in May, which is 25 clips. In total, she sold 50 + 25 = 75 clips.  
**Answer:** 75<｜end▁of▁sentence｜>


In [53]:
prompt = """
    <｜begin▁of▁sentence｜>You are a helpful assistant. You should generate answer as short as possible.
    <｜User｜>Natalia sold clips to 50 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
"""

# 编码，推理
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=False,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)
response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
print(response)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


</think>

Natalia sold clips in April to 50 friends and half as many in May. Therefore, she sold 25 clips in May. In total, she sold 50 + 25 = 75 clips in April and May.

Answer: \boxed{75}
